# Carla Segmentation

Image Segmentation on Carla Dataset

### Import Libraries

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as pimg
import imageio
import cv2
import scipy
import albumentations as albu
import os
import random

# PyTorch Libraries
!pip3 install segmentation-models-pytorch torchinfo
import torch as tr
from torch.utils.data import DataLoader, Dataset
from segmentation_models_pytorch.utils import base
from torchinfo import summary
import segmentation_models_pytorch as smp

## Segmentation Model

In [2]:
Model = smp.Unet(
    encoder_name="resnet34",
    encoder_weights="imagenet",
    classes=13,
    activation='softmax2d'
)

## Data Preparation and Processing

### Importing and Visualizing Data

Paths of Images and Masks

In [3]:
# Image and Masks Directories
# Kaggle
Images_Directories = ["../input/lyft-udacity-challenge/"+"data"+i+"/"+"data"+i+"/CameraRGB/" for i in ['A', 'B', 'C', 'D', 'E']]
Masks_Directories = ["../input/lyft-udacity-challenge/"+"data"+i+"/"+"data"+i+"/CameraSeg/" for i in ['A', 'B', 'C', 'D', 'E']]

# Filenames of Images/Masks
def Listing_Images(Directories):
    Images_Path = []
    for directory in range(len(Directories)):
        image_filenames = os.listdir(Directories[directory])
        for image_filename in image_filenames:
            Images_Path.append(Directories[directory] + image_filename)
    return Images_Path

# Paths for Images
Images_Path = Listing_Images(Images_Directories)

# Paths for Masks
Masks_Path = Listing_Images(Masks_Directories)

# Number of Images or Masks
N = len(Masks_Path)

Visualizing Images and Masks from Dataset

In [4]:
for i in range(4):
    # Selecting a random index
    ind = np.random.randint(N)

    # Reading an Image and its Mask
    img = imageio.imread(Images_Path[ind])
    mask = imageio.imread(Masks_Path[ind])

    # Displaying Images
    plt.figure(figsize=(12,6))
    plt.subplot(1,2,1)
    plt.title("Image")
    plt.imshow(img)
    plt.subplot(1,2,2)
    plt.title("Mask")
    plt.imshow(np.max(mask, axis=-1), cmap='jet')
    plt.show()

### Train and Test Split

In [5]:
Train_Ind = np.random.randint(N, size=int(0.8*N))
Valid_Ind = list(set(np.arange(N)) - set(Train_Ind))

# Train Data Path
Train_Images_Path = np.array(Images_Path)[Train_Ind]
Train_Masks_Path = np.array(Masks_Path)[Train_Ind]

# Valid Data Path
Valid_Images_Path = np.array(Images_Path)[Valid_Ind]
Valid_Masks_Path = np.array(Masks_Path)[Valid_Ind]

## Augmentations and Preprocessing

Augmentation Function

In [6]:
def Create_Augmentation_Function():
    train_transform = [
        albu.ShiftScaleRotate(scale_limit=0.1, rotate_limit=0., shift_limit=0.1, p=1, border_mode=0),

        albu.IAAAdditiveGaussianNoise(p=0.2),

        albu.OneOf(
            [
                albu.CLAHE(p=1),
                albu.RandomBrightness(p=1),
                albu.RandomGamma(p=1),
            ],
            p=0.6,
        ),

        albu.OneOf(
            [
                albu.IAASharpen(p=1),
                albu.Blur(blur_limit=3, p=1),
                albu.MotionBlur(blur_limit=3, p=1),
            ],
            p=0.6,
        ),

        albu.OneOf(
            [
                albu.RandomContrast(p=1),
                albu.HueSaturationValue(p=1),
            ],
            p=0.6,
        ),
    ]
    return albu.Compose(train_transform)


Augmentations = Create_Augmentation_Function()

Preprocessing Function

In [7]:
Preprocess = smp.encoders.get_preprocessing_fn("resnet34", "imagenet")

def to_tensor(x, **kwargs):
    return x.transpose(2, 0, 1).astype('float32')

def Preprocessing(Preprocess_Function):   
    _transform = [
        albu.Lambda(image=Preprocess_Function),
        albu.Lambda(image=to_tensor),
    ]
    return albu.Compose(_transform)

### Dataset and DataLoaders

Dataset

In [8]:
class CarlaSegmentationDataset(Dataset):
    def __init__(self, Images_Path, Masks_Path, num_classes, Augmentation, Preprocess):
        self.Images_Path = Images_Path
        self.Masks_Path = Masks_Path
        self.num_classes = num_classes
        self.augmentation = Augmentation
        self.preprocessing = Preprocess
        
    def __getitem__(self, i):
        try:
            image = imageio.imread(self.Images_Path[i])
            mask = imageio.imread(self.Masks_Path[i])
        except:
            image = imageio.imread(self.Images_Path[i].decode("utf-8"))
            mask = imageio.imread(self.Masks_Path[i].decode("utf-8"))
        
        # Resizing
        image = cv2.resize(image, (512,256))
        mask = cv2.resize(mask, (512,256))
        
        mask = np.max(mask, axis=-1)
        mask = np.eye(self.num_classes)[mask]
        
        if self.augmentation:
            # Applying Augmentations
            Augmented_Datapoint = self.augmentation(image=image, mask=mask)
            image, mask = Augmented_Datapoint['image'], Augmented_Datapoint['mask']
            
        if self.preprocessing:
            # Applying Preprocessing
            Preprocessed_Datapoint = self.preprocessing(image=image, mask=mask)
            image, mask = Preprocessed_Datapoint['image'], Preprocessed_Datapoint['mask']
            mask = np.transpose(mask, (2,0,1))
        
        return image, tr.LongTensor(mask)
    
    def __len__(self):
        return len(self.Images_Path)

DataLoader

In [9]:
Train_Dataset = CarlaSegmentationDataset(Train_Images_Path, Train_Masks_Path, 13, Augmentations, Preprocessing(Preprocess))
Valid_Dataset = CarlaSegmentationDataset(Valid_Images_Path, Valid_Masks_Path, 13, None, Preprocessing(Preprocess))

Visualizing Images and Masks after Augmentation and Preprocessing

In [10]:
for i in range(3):
    image, mask = Train_Dataset[i]
    
    # Displaying Image
    plt.figure(figsize=(12,6))

    plt.subplot(1,2,1)
    plt.title("Input Image")
    plt.imshow(image.transpose(1,2,0))

    plt.subplot(1,2,2)
    plt.title("True Mask")
    plt.imshow(np.argmax(mask, axis=0), cmap='jet')

    plt.show()

## Training Segmentation Model

Training Parameters


In [11]:
Batch_Size = 16
Epochs = 5

DataLoaders

In [12]:
Train_DataLoader = DataLoader(Train_Dataset, batch_size=Batch_Size, shuffle=True)
Valid_DataLoader = DataLoader(Valid_Dataset, batch_size=Batch_Size, shuffle=True)

Training Settings

In [13]:
optimizer = tr.optim.Adam(Model.parameters(), lr=0.001)

class LossFunction(base.Loss):
    def __init__(self, **kwargs):
        super().__init__(**kwargs)
        self.Loss = smp.utils.losses.CrossEntropyLoss()
        
    def forward(self, y_pr, y_gt):
        y_gt = tr.argmax(y_gt, axis=1)
        return self.Loss(y_pr, y_gt)
    
Accuracy = smp.utils.metrics.Accuracy()
IoU = smp.utils.metrics.IoU()
Loss = LossFunction()

Training and Validation Epochs

In [14]:
train_epoch = smp.utils.train.TrainEpoch(
    Model,
    loss=Loss,
    metrics=[Accuracy, IoU],
    optimizer=optimizer,
    device="cuda",
    verbose=True
)

valid_epoch = smp.utils.train.ValidEpoch(
    Model, 
    loss=Loss, 
    metrics=[Accuracy, IoU],
    device="cuda",
    verbose=True
)

Training the Model

In [15]:
for epoch in range(Epochs):
    print ("Epoch-" + str(epoch) + ":")
    train_epoch.run(Train_DataLoader)
    valid_epoch.run(Valid_DataLoader)

Saving Model

In [16]:
tr.save(Model, "Segmentation_Model.pth")

## Evaluate Model

In [17]:
for i in range(3):
    image, mask = Valid_Dataset[i]
    
    # Predicting
    X = tr.from_numpy(np.expand_dims(image, axis=0)).to("cuda")
    pred = Model.predict(X).cpu().numpy()[0]
    
    # Displaying Image
    plt.figure(figsize=(18,6))

    plt.subplot(1,3,1)
    plt.title("Input Image")
    plt.imshow(image.transpose(1,2,0))

    plt.subplot(1,3,2)
    plt.title("True Mask")
    plt.imshow(np.argmax(mask, axis=0), cmap='jet')
    
    plt.subplot(1,3,3)
    plt.title("Pred Mask")
    plt.imshow(np.argmax(pred, axis=0), cmap='jet')

    plt.show()